[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/chatbots/nemo-guardrails/02a-actions-weather-api.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/generation/chatbots/nemo-guardrails/02a-actions-weather-api.ipynb)

In [11]:
!pip install -qU \
    nemoguardrails==0.4.0 \
    openai==0.27.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 39.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Set your OpenAI API key:

In [12]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

# Adding Actions: Weather API

In Guardrails we can add _actions_ to our colang files. These actions allow us to execute code, for example:

```
define flow
    user ask question
    $answer = execute qa_func(prompt=$last_user_message)
    bot $answer
```

In this Colang flow we expect the user to ask some question (`user ask question`), if so we run an _action_ (equivalent to a Python function) called `qa_func`, we also pass the `$last_user_message` (a default variable set by Guardrails) to this function via the function's `prompt` parameter.

The function/action returns a value which we store in the `$answer` context variable, we then tell the bot to return this answer to the user (`bot $answer`).

Let's take a look at an example colang file for this:

In [43]:
colang_content = """
# define limits
define user ask weather
    "how is the weather today?"
    "should I wear a coat?"

define bot answer weather
    bot report weather

define flow weather
    user ask weather
    $coords = execute location_api()
    $weather = execute weather_api(coords=$coords)
    bot answer weather

# here we use the chatbot for anything else
define flow
    user ...
    bot greeting
"""
yaml_content = """
models:
- type: main
  engine: openai
  model: text-davinci-003
"""

In our Colang we specify the `weather` dialogue flow with two calls to actions, one for a location API and the next for a weather API (using info from the location API).

In [44]:
import requests

async def weather_api(coords: list):
    latitude, longitude = coords
    res = requests.get(
        "https://api.open-meteo.com/v1/forecast",
        params={
            "latitude": latitude,
            "longitude": longitude,
            "current_weather": "true"
        }
    )
    weather = res.json()["current_weather"]
    weather_report = f"""The current weather is:
    temperature: {weather["temperature"]}
    windspeed: {weather["windspeed"]}
    wind direction: {weather["winddirection"]} degrees
    And it is {"daytime" if weather["is_day"] else "nightime"}"""
    return weather_report

async def location_api():
    res = requests.get("http://ip-api.com/json/")
    return res.json()['lat'], res.json()['lon']

We've defined our location and weather APIs required for our bot to give us up to date advice on the weather in our current location.

In [45]:
from nemoguardrails import LLMRails, RailsConfig

# initialize rails config
config = RailsConfig.from_content(
    colang_content=colang_content,
    yaml_content=yaml_content
)
# create rails
rails = LLMRails(config, verbose=True)

We need to register the two functions as actions like so:

In [46]:
rails.register_action(action=location_api, name="location_api")
rails.register_action(action=weather_api, name="weather_api")

Let's try our guardrails...

In [47]:
await rails.generate_async(prompt="hello")

'Hi there! How can I be of assistance?'

Now we can try again:

In [48]:
await rails.generate_async(prompt="how is the weather?")

'The current weather in Washington, DC is 27.8 degrees, with a wind speed of 13.7 mph and a wind direction of 275.0 degrees. It is currently daytime.'

In [49]:
await rails.generate_async(prompt="do I need a sweatshirt today?")

'It is currently 27.8 degrees outside with a windspeed of 13.7 and a wind direction of 275.0 degrees. Based on this, it is recommended that you wear a sweatshirt today.'

Great, we can see our weather checking Guardrails are working!